<a href="https://colab.research.google.com/github/namhwui/MixtureAlgos_py/blob/main/MixtureSAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementing finite mixture of shifted asymmetric Laplace distributions\
From 'Mixtures of Shifted Asymmetric Laplace Distributions' \
by B.C. Franczak, R.P. Browne, P.D. McNicholas\
arxiv: https://arxiv.org/abs/1207.1727

In [ ]:
import numpy as np
from numpy.linalg import inv
from numpy.linalg import det
from scipy.special import kv
from scipy.spatial import distance

def mahala_squared(x, invsigma, mu=None):
  # computes squared mahalanobis distance of a n-by-p array x
  if mu is None:
    mu = np.zeros(invsigma.shape[0])
  
  n = 1
  if len(x.shape) == 2:
    n = x.shape[0]
  
  x_mu = x - mu
  val = np.zeros(n)
  for ii in range(0, n):
    val[ii] = np.transpose(x_mu[ii, :]) @ invsigma @ x_mu[ii, :]
  return(val)



def component_density(x, param):
  # computes shifted asymmetric laplace density

  # compute required bilinear forms
  xSx = mahala_squared(x, param['invsigma'], param['mu'])
  aSa = np.transpose(param['alpha']) @ param['invsigma'] @ param['alpha']
  exp_xSa = np.exp((x - param['mu']) @ param['invsigma'] @ param['alpha'])
  const = np.sqrt(det(2 * np.pi * param['sigma']))
  u = np.sqrt((2 + aSa) * xSx)
  val = (2 * exp_xSa/const) * (xSx / (2 + aSa))**(param['nu']/2) * kv(param['nu'], u)
  return(val) 


def mixture_density(x, model, logged = False, post = False):
  # computes density of G-component Gaussian mixture model
  # if logged = True, log-density is returned
  # if logged = False, and post = True, then a length-G array of component-wise
  #   posterior membership probabilities is returned
  G = model['G']
  param = model['param']
  n = 1
  if len(x.shape) == 2:
    n = x.shape[0]
  val = np.zeros([n, G]) # Allocating an array of appropriate length a priori is better than iterative appending, like in R

  for g in range(0, G):
    #print(component_density(x, param[g]))
    val[:, g] = model['prop'][g] * component_density(x, param[g])

  if logged:
    return(np.log(np.sum(val, axis = 1))) # np.sum is faster on numpy arrays, but sum is faster on lists
  elif post:
    return(val / np.sum(val, axis = 1))
  else:
    return(np.sum(val, axis = 1))

#def expect(x, param):


x = np.array([[1, 2], [1, 3], [1, 0]])
param = [{'mu': np.array([1, 1]),
         'alpha': np.array([2, 2]),
         'sigma': np.array([[1, 0], [0, 1]]),
         'invsigma': np.array([[1, 0], [0, 1]]),
         'nu':2},
         {'mu': np.array([0, 0]),
         'alpha': np.array([-2, -2]),
         'sigma': np.array([[1, 0], [0, 1]]),
         'invsigma': np.array([[1, 0], [0, 1]]),
         'nu':4}]

model = {'prop':np.array([0.5, 0.5]),
         'param':param}
model['G'] = len(model['prop'])
#print(model['param'][0])
#mixture_density(x, model)










array([0.00585003, 0.00408424, 0.0001571 ])